In [1]:
%matplotlib inline

import glob
import os
import json
import pysam
import gspread
from IPython.core.display import HTML
import numpy as np
from oauth2client.client import SignedJwtAssertionCredentials
import pandas as pd
import pybedtools
import seaborn as sns
import json
import pybedtools
from gscripts.general import region_helpers
from gscripts.general import dataviz
import matplotlib as mpl
import datetime
img_dir ="/home/gpratt/Dropbox/encode_integration/for_eric/idr_qc/"

from gscripts.encode import encode_helpers
import functools
from tqdm import tqdm, tqdm_notebook
tqdm.pandas("Progress: ")
from gscripts import qtools
RESET = False

In [2]:
legend = None

OUTSIDE_LEGEND_SAVEFIG_KWS = dict(bbox_extra_artists=(legend,),
                                  bbox_inches='tight')
from matplotlib import rc

mpl.rcParams['svg.fonttype'] = 'none'

rc('text', usetex=False)
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})

In [3]:
input_norm_dir = "/projects/ps-yeolab3/encode/analysis/Eric_Input_Norm/"
split_dir = "/home/gpratt/projects/idr/analysis/idr_v2/"
out_dir = "/home/gpratt/projects/encode/analysis/peak_reanalysis_v14/"
frip_out_dir = "/home/gpratt/projects/encode/analysis/frip_calculations/"
downsample_path = "/home/gpratt/projects/idr/analysis/downsample_v2/"

In [4]:
merged_data = encode_helpers.get_merged_data()

#Filter out anything sequenced after 16/7/18
merged_data = merged_data[merged_data['Submitted Date'] < datetime.date(2016, 7, 18)]
#Filter out anything not_qced
merged_data = merged_data[merged_data.is_qced]

In [5]:
#Construct manifest to process data
merged_data['genome'] = 'hg19'
idr_manifest = merged_data.unstack()
idr_manifest = idr_manifest.reset_index()
idr_manifest = idr_manifest[[("CLIP", "rep1"), ("CLIP", "rep2"),
                             ("input_norm", "rep1"), ("input_norm", "rep2"), 
                             ("INPUT", "rep1"),
                             ("genome", "rep1"),
                             ("uID", ""), 
                             ("submitted", "rep1"),
                             ("peaks_submitable", "rep1")]]

In [6]:
em_peaks_dir = "/home/gpratt/projects/idr/analysis/input_em_peaks"
idr_manifest[('em_norm', 'rep1')] = idr_manifest[('input_norm', 'rep1')].apply(lambda x: os.path.join(em_peaks_dir, ".".join(os.path.basename(x).split(".")[:2]) + ".peaks.em.v3.bed"))
idr_manifest[('em_norm', 'rep2')] = idr_manifest[('input_norm', 'rep2')].apply(lambda x: os.path.join(em_peaks_dir, ".".join(os.path.basename(x).split(".")[:2]) + ".peaks.em.v3.bed"))

In [7]:
def get_split_bed(fn, split_num):
    fn = ".".join(os.path.basename(fn).split(".")[:3])
    fn = "{}.{}.peaks.norm.compressed.l2Fixed.bed".format(fn, split_num)
    fn = os.path.join(split_dir, fn)
    return fn

split_01 = functools.partial(get_split_bed, split_num="01")
split_02 = functools.partial(get_split_bed, split_num="02")

def get_split_bam(fn, split_num):
    fn = ".".join(os.path.basename(fn).split(".")[:3])
    fn = "{}.{}.bam".format(fn, split_num)
    fn = os.path.join(split_dir, fn)
    return fn

split_bam_01 = functools.partial(get_split_bam, split_num="01")
split_bam_02 = functools.partial(get_split_bam, split_num="02")

In [8]:
idr_manifest[('rep1', '01')] = idr_manifest[('CLIP', 'rep1')].apply(split_01)
idr_manifest[('rep1', '02')] = idr_manifest[('CLIP', 'rep1')].apply(split_02)

idr_manifest[('rep2', '01')] = idr_manifest[('CLIP', 'rep2')].apply(split_01)
idr_manifest[('rep2', '02')] = idr_manifest[('CLIP', 'rep2')].apply(split_02)

idr_manifest[('merged', '01')] = idr_manifest['uID'].apply(lambda x: os.path.join(split_dir, x + ".merged.01.peaks.norm.compressed.l2Fixed.bed"))
idr_manifest[('merged', '02')] = idr_manifest['uID'].apply(lambda x: os.path.join(split_dir, x + ".merged.02.peaks.norm.compressed.l2Fixed.bed"))

idr_manifest[('rep1', '01_bam')] = idr_manifest[('CLIP', 'rep1')].apply(split_bam_01)
idr_manifest[('rep1', '02_bam')] = idr_manifest[('CLIP', 'rep1')].apply(split_bam_02)

idr_manifest[('rep2', '01_bam')] = idr_manifest[('CLIP', 'rep2')].apply(split_bam_01)
idr_manifest[('rep2', '02_bam')] = idr_manifest[('CLIP', 'rep2')].apply(split_bam_02)

idr_manifest[('merged', '01_bam')] = idr_manifest['uID'].apply(lambda x: os.path.join(split_dir, x + ".merged.01.bam"))
idr_manifest[('merged', '02_bam')] = idr_manifest['uID'].apply(lambda x: os.path.join(split_dir, x + ".merged.02.bam"))

In [9]:
idr_manifest = idr_manifest[idr_manifest[('rep1', '01')].apply(os.path.exists)]
idr_manifest = idr_manifest[idr_manifest[('rep1', '02')].apply(os.path.exists)]
idr_manifest = idr_manifest[idr_manifest[('rep2', '01')].apply(os.path.exists)]
idr_manifest = idr_manifest[idr_manifest[('rep2', '02')].apply(os.path.exists)]
idr_manifest = idr_manifest[idr_manifest[('merged', '01')].apply(os.path.exists)]
idr_manifest = idr_manifest[idr_manifest[('merged', '02')].apply(os.path.exists)]

idr_manifest = idr_manifest[idr_manifest[('rep1', '01_bam')].apply(os.path.exists)]
idr_manifest = idr_manifest[idr_manifest[('rep1', '02_bam')].apply(os.path.exists)]
idr_manifest = idr_manifest[idr_manifest[('rep2', '01_bam')].apply(os.path.exists)]
idr_manifest = idr_manifest[idr_manifest[('rep2', '02_bam')].apply(os.path.exists)]
idr_manifest = idr_manifest[idr_manifest[('merged', '01_bam')].apply(os.path.exists)]
idr_manifest = idr_manifest[idr_manifest[('merged', '02_bam')].apply(os.path.exists)]

In [10]:
len(idr_manifest)

283

# Run "Em" on all the RBPs

In [15]:
# def run_em_input_norm(peak, ip_bam, input_bam):
#     em_peak_dir = "/home/gpratt/projects/idr/analysis/input_em_peaks"
#     bed = os.path.join(em_peak_dir, ".".join(os.path.basename(peak).split(".")[:3]) + ".em.bed")
#     full = os.path.join(em_peak_dir, ".".join(os.path.basename(peak).split(".")[:3]) + ".em.full")

#     return "eclip_input_norm -p {peak} --ip_bam {ip_bam} --input_bam {input_bam} -o {out_bed} -f {out_full}".format(peak=peak,
#                                                                                                             ip_bam=ip_bam,
#                                                                                                             input_bam=input_bam,
#                                                                                                             out_bed=bed,
#                                                                                                             out_full=full)

# def row_based_em_input_norm(row, rep, split):
#     peaks = row[rep, split]
#     ip_bam = row[rep, split + "_bam"]
#     input_bam = row['INPUT', 'rep1'] 
#     return run_em_input_norm(peaks, ip_bam, input_bam)

In [16]:
# rep1_01_norm = functools.partial(row_based_em_input_norm, rep="rep1", split="01")
# rep1_02_norm = functools.partial(row_based_em_input_norm, rep="rep1", split="02")

# rep2_01_norm = functools.partial(row_based_em_input_norm, rep="rep2", split="01")
# rep2_02_norm = functools.partial(row_based_em_input_norm, rep="rep2", split="02")

# merged_01_norm = functools.partial(row_based_em_input_norm, rep="merged", split="01")
# merged_02_norm = functools.partial(row_based_em_input_norm, rep="merged", split="02")

In [17]:
# em_input_norm_cmds = np.concatenate([idr_manifest[~idr_manifest[('rep1', '01')].apply(em_peaks).apply(os.path.exists)].apply(rep1_01_norm, axis=1).values,
#                                      idr_manifest[~idr_manifest[('rep1', '02')].apply(em_peaks).apply(os.path.exists)].apply(rep1_02_norm, axis=1).values,
#                                      idr_manifest[~idr_manifest[('rep2', '01')].apply(em_peaks).apply(os.path.exists)].apply(rep2_01_norm, axis=1).values,
#                                      idr_manifest[~idr_manifest[('rep2', '02')].apply(em_peaks).apply(os.path.exists)].apply(rep2_02_norm, axis=1).values,
#                                      idr_manifest[~idr_manifest[('merged', '01')].apply(em_peaks).apply(os.path.exists)].apply(merged_01_norm, axis=1).values,
#                                      idr_manifest[~idr_manifest[('merged', '02')].apply(em_peaks).apply(os.path.exists)].apply(merged_02_norm, axis=1).values,])

In [19]:
# len(em_input_norm_cmds)

In [20]:
# job_name = "em_input_norm_idr"
# job = qtools.Submitter(commands=em_input_norm_cmds,
#                  job_name="{}".format(job_name),
#                 sh_filename="/home/gpratt/projects/idr/scripts/{}.sh".format(job_name),
#                 array=True,
#                 walltime="8:00:00",
#                 out_filename="/home/gpratt/projects/idr/scripts/{}.out".format(job_name),
#                 err_filename="/home/gpratt/projects/idr/scripts/{}.err".format(job_name),
#                 queue="home-yeo")
# job.job()

# print "/home/gpratt/projects/idr/scripts/{}.sh".format(job_name)

# Get the old entropy peaks

In [21]:
def calculate_entropy(row, total_ip_reads, total_input_reads):
    p_ip = float(row.ip_reads) / total_ip_reads
    p_input = float(row.input_reads) / total_input_reads
    
    return p_ip * np.log2(p_ip / p_input)

def calculate_entropy_file(peaks, ip_bam, input_bam):
    entropy_peak_dir = "/home/gpratt/projects/idr/analysis/idr_v2_entropy_bed"
    full_fn = peaks + '.full'
    peak_name, ext = os.path.splitext(peaks)
    out_bed = os.path.join(entropy_peak_dir, os.path.basename(peak_name) + ".entropy.bed")
    
    if os.path.exists(out_bed):
        return out_bed
    
    ip_reads = pysam.Samfile(ip_bam).mapped
    input_reads = pysam.Samfile(input_bam).mapped

    read_counts = pd.read_table(full_fn, names=["chrom", "start", "stop", "full_name", 
                                                  "ip_reads", "input_reads", "p_val",
                                                  "chiqc_score", "test", "enrichment", 
                                                  "log10_p_val", "log2_fold_change"])

    tool = functools.partial(calculate_entropy, total_ip_reads=ip_reads, total_input_reads=input_reads)
    read_counts['entropy'] = read_counts.apply(tool, axis=1)
    read_counts['strand'] = read_counts.full_name.apply(lambda x: x.split(":")[2])
    read_counts = read_counts[['chrom', 'start', 'stop', 'log10_p_val', 'log2_fold_change', 'strand', 'entropy', 'entropy', 'entropy']]
    read_counts.to_csv(out_bed, index=False, header=False, sep="\t")
    return out_bed

def row_based_calculate_entropy_file(row, rep, split):
    peaks = row[rep, split]
    peaks = ".".join(peaks.split(".")[:-3]) + ".bed"
    
    ip_bam = row[rep, split + "_bam"]
    input_bam = row['INPUT', 'rep1'] 
    return calculate_entropy_file(peaks, ip_bam, input_bam)

def row_based_calculate_entropy_real_file(row, rep, split):
    peaks = row['input_norm', split]
    peaks = ".".join(peaks.split(".")[:2]) + ".peaks.l2inputnormnew.bed.full.compressed2.bed"
    
    ip_bam = row[rep, split]
    input_bam = row['INPUT', 'rep1'] 
    return calculate_entropy_file(peaks, ip_bam, input_bam)

In [22]:
rep1_01_norm = functools.partial(row_based_calculate_entropy_file, rep="rep1", split="01")
rep1_02_norm = functools.partial(row_based_calculate_entropy_file, rep="rep1", split="02")

rep2_01_norm = functools.partial(row_based_calculate_entropy_file, rep="rep2", split="01")
rep2_02_norm = functools.partial(row_based_calculate_entropy_file, rep="rep2", split="02")

merged_01_norm = functools.partial(row_based_calculate_entropy_file, rep="merged", split="01")
merged_02_norm = functools.partial(row_based_calculate_entropy_file, rep="merged", split="02")

rep1_norm = functools.partial(row_based_calculate_entropy_real_file, rep="CLIP", split="rep1")
rep2_norm = functools.partial(row_based_calculate_entropy_real_file, rep="CLIP", split="rep2")

In [23]:
len(idr_manifest)

283

374 don't remember if we are analyzing using 4000 or regular

In [25]:
idr_manifest[('rep1', '01_entropy')] = idr_manifest.progress_apply(rep1_01_norm, axis=1)
idr_manifest[('rep1', '02_entropy')] = idr_manifest.progress_apply(rep1_02_norm, axis=1)
idr_manifest[('rep2', '01_entropy')] = idr_manifest.progress_apply(rep2_01_norm, axis=1)
idr_manifest[('rep2', '02_entropy')] = idr_manifest.progress_apply(rep2_02_norm, axis=1)
idr_manifest[('merged', '01_entropy')] = idr_manifest.progress_apply(merged_01_norm, axis=1)
idr_manifest[('merged', '02_entropy')] = idr_manifest.progress_apply(merged_02_norm, axis=1)
idr_manifest[('real', '01_entropy')] = idr_manifest.progress_apply(rep1_norm, axis=1)
idr_manifest[('real', '02_entropy')] = idr_manifest.progress_apply(rep2_norm, axis=1)

283it [00:00, 1790.53it/s]            
283it [00:00, 1912.11it/s]            
283it [00:00, 1850.34it/s]            
283it [00:00, 1891.33it/s]            
283it [00:00, 1918.48it/s]            
283it [00:00, 1894.89it/s]            
115it [00:00, 275.23it/s]             

IOError: ('File /projects/ps-yeolab3/encode/analysis/Eric_Input_Norm/374_01.basedon_374_01.peaks.l2inputnormnew.bed.full.compressed2.bed.full does not exist', u'occurred at index 96')

194it [00:20, 275.23it/s]

In [325]:
idr_out_dir = "/home/gpratt/projects/idr/analysis/idr_v3"

def idr(peak_1, peak_2, out_file):
    return "source activate py3 && idr  '--samples'  '{}'  '{}'  '--input-file-type' 'bed'  '--rank' '7'  '--peak-merge-method' 'max'  '--plot'  '-o' '{}' '--allow-negative-scores'".format(peak_1, peak_2, out_file)

def rep1_idr(row):
    peak_1 = row['rep1', '01_entropy']
    peak_2 = row['rep1', '02_entropy']
    out_file = row.uID[0] + ".rep1.txt"
    out_file = os.path.join(idr_out_dir, out_file)
    return idr(peak_1, peak_2, out_file)

def rep2_idr(row):
    peak_1 = row['rep2', '01_entropy']
    peak_2 = row['rep2', '02_entropy']
    out_file = row.uID[0] + ".rep2.txt"
    out_file = os.path.join(idr_out_dir, out_file)
    return idr(peak_1, peak_2, out_file)

def merged_idr(row):
    peak_1 = row['merged', '01_entropy']
    peak_2 = row['merged', '02_entropy']
    out_file = row.uID[0] + ".combined.txt"
    out_file = os.path.join(idr_out_dir, out_file)
    return idr(peak_1, peak_2, out_file)

def real_idr(row):
    peak_1 = row['real', '01_entropy']
    peak_2 = row['real', '02_entropy']
    out_file = row.uID[0] + ".txt"
    out_file = os.path.join(idr_out_dir, out_file)
    return idr(peak_1, peak_2, out_file)


In [329]:
idr_cmds = np.concatenate([idr_manifest.apply(rep1_idr, axis=1).values,
                                     idr_manifest.apply(rep2_idr, axis=1).values,
                                     idr_manifest.apply(merged_idr, axis=1).values,
                                    ])                                     

In [358]:
idr_cmds = idr_manifest.apply(real_idr, axis=1).values

In [359]:
job_name = "idr"
job = qtools.Submitter(commands=idr_cmds,
                 job_name="{}".format(job_name),
                sh_filename="/home/gpratt/projects/idr/scripts/{}.sh".format(job_name),
                array=True,
                walltime="8:00:00",
                out_filename="/home/gpratt/projects/idr/scripts/{}.out".format(job_name),
                err_filename="/home/gpratt/projects/idr/scripts/{}.err".format(job_name),
                queue="home-yeo")
job.job()

print "/home/gpratt/projects/idr/scripts/{}.sh".format(job_name)

/home/gpratt/projects/idr/scripts/idr.sh


running 282 tasks as an array-job.


# Get the rest of the entropy peaks

In [157]:
em_peaks_dir = "/home/gpratt/projects/idr/analysis/input_em_peaks"

def em_peaks(peak):
    bed = os.path.join(em_peaks_dir, ".".join(os.path.basename(peak).split(".")[:3]) + ".em.v3.bed")
    return bed

In [211]:
idr_manifest[('rep1', '01_peaks')] = idr_manifest[('rep1', '01')].apply(em_peaks).apply(adjust_entropy)
idr_manifest[('rep1', '02_peaks')] = idr_manifest[('rep1', '02')].apply(em_peaks).apply(adjust_entropy)

idr_manifest[('rep2', '01_peaks')] = idr_manifest[('rep2', '01')].apply(em_peaks).apply(adjust_entropy)
idr_manifest[('rep2', '02_peaks')] = idr_manifest[('rep2', '02')].apply(em_peaks).apply(adjust_entropy)

idr_manifest[('merged', '01_peaks')] = idr_manifest[('merged', '01')].apply(em_peaks).apply(adjust_entropy)
idr_manifest[('merged', '02_peaks')] = idr_manifest[('merged', '02')].apply(em_peaks).apply(adjust_entropy)

ValueError: File "/home/gpratt/projects/idr/analysis/input_em_peaks/202_01_PTBP1.merged.r2.em.v3.bed" does not exist

In [228]:
idr_out_dir = "/home/gpratt/projects/idr/analysis/idr_v3"

def idr(peak_1, peak_2, out_file):
    return "source activate py3 && idr  '--samples'  '{}'  '{}'  '--input-file-type' 'bed'  '--rank' '7'  '--peak-merge-method' 'max'  '--plot'  '-o' '{}' '--allow-negative-scores'".format(peak_1, peak_2, out_file)

def rep1_idr(row):
    peak_1 = row['rep1', '01_peaks']
    peak_2 = row['rep1', '02_peaks']
    out_file = row.uID[0] + ".rep1.txt"
    out_file = os.path.join(idr_out_dir, out_file)
    return idr(peak_1, peak_2, out_file)

def rep2_idr(row):
    peak_1 = row['rep2', '01_peaks']
    peak_2 = row['rep2', '02_peaks']
    out_file = row.uID[0] + ".rep2.txt"
    out_file = os.path.join(idr_out_dir, out_file)
    return idr(peak_1, peak_2, out_file)

def merged_idr(row):
    peak_1 = row['merged', '01_peaks']
    peak_2 = row['merged', '02_peaks']
    out_file = row.uID[0] + ".combined.txt"
    out_file = os.path.join(idr_out_dir, out_file)
    return idr(peak_1, peak_2, out_file)

def real_idr(row):
    peak_1 = row['em_norm', 'rep1_adjust']
    peak_2 = row['em_norm', 'rep2_adjust']
    out_file = row.uID[0] + ".txt"
    out_file = os.path.join(idr_out_dir, out_file)
    return idr(peak_1, peak_2, out_file)


In [ ]:
/home/elvannostrand/data/clip/CLIPseq_analysis/scripts/PIPELINE_EricVersion/compare_vs_input_peakbased_wrapper2_OneOrTwoRepVersion_PEbamfileversion_submit.pl i

In [222]:
def adjust_entropy_interval(interval):
    interval[4] = interval[6]
    interval[3] = interval[6]
    interval.append(interval[6])
    interval.append(interval[6])
    return interval

def adjust_entropy(fn):
    out_name, ext = os.path.splitext(fn)
    out_file = out_name + ".adjust" + ext
    if os.path.exists(out_file):
        return out_file
    bedtool = pybedtools.BedTool(fn)
    bedtool.each(adjust_entropy_interval).saveas(out_file)
    return out_file

In [223]:
idr_manifest = idr_manifest[idr_manifest[('em_norm', 'rep1')].apply(os.path.exists)]
idr_manifest = idr_manifest[idr_manifest[('em_norm', 'rep2')].apply(os.path.exists)]

In [225]:
idr_manifest[('em_norm', 'rep1_adjust')] = idr_manifest[('em_norm', 'rep1')].progress_apply(adjust_entropy)
idr_manifest[('em_norm', 'rep2_adjust')] = idr_manifest[('em_norm', 'rep2')].progress_apply(adjust_entropy)





  0%|          | 0/282 [00:00<?, ?it/s]



 39%|███▉      | 111/282 [00:00<00:00, 1101.40it/s]



 86%|████████▌ | 242/282 [00:00<00:00, 1151.18it/s]



100%|██████████| 282/282 [00:00<00:00, 1258.81it/s]



  0%|          | 0/282 [00:00<?, ?it/s]



 76%|███████▌  | 214/282 [00:00<00:00, 2137.09it/s]



100%|██████████| 282/282 [00:00<00:00, 2109.97it/s]

In [230]:
idr_manifest.apply(real_idr, axis=1)[1]

"source activate py3 && idr  '--samples'  '/home/gpratt/projects/idr/analysis/input_em_peaks/203_01.basedon_203_01.peaks.em.v3.adjust.bed'  '/home/gpratt/projects/idr/analysis/input_em_peaks/203_02.basedon_203_02.peaks.em.v3.adjust.bed'  '--input-file-type' 'bed'  '--rank' '7'  '--peak-merge-method' 'max'  '--plot'  '-o' '/home/gpratt/projects/idr/analysis/idr_v3/203.txt' '--allow-negative-scores'"